In [ ]:
;pwd

In [ ]:
using Distributions

In [ ]:
srand(1234)
nTr = 2
n,p = 10,100
Q = convert(Array{Float64,2},rand(Binomial(2,0.5),n,p))
α2 = randn(p)
α1 = randn(p)*2 + α2
Alpha = [α1 α2]'
a1 = Q*α1
a2 = Q*α2;
R = cov([a1 a2])
R0 = R
L  = chol(R0)
e  = L'*randn(2,size(Q,1))
Y = Alpha*Q' + e
G = cov([α1 α2])
Q = Q - ones(n)*mean(Q,1)
G0 = G

In [ ]:
YCorr = copy(Y) #- Alpha*Q'
μ = zeros(nTr,1)

In [ ]:
νG = 10 + nTr
νR = 10 + nTr
PG  = G*(νG - nTr - 1)
PR  = R*(νR - nTr - 1)

In [ ]:
niter = 5000
GMean = zeros(nTr,nTr)
RMean = zeros(nTr,nTr)
Alpha = zeros(nTr,p);

In [ ]:
srand(1234)
for iter = 1:niter
    iR = inv(R)
    iG = inv(G)
    # sample μ
    # unadjust for mu
    YCorr = YCorr .+ μ
    Lhs = n*iR
    Rhs = iR * sum(YCorr,2)
    for i=1:nTr
        lhs = Lhs[i,i]
        ilhs = 1/lhs
        rhs = (Rhs[i] - Lhs[i,:]*μ)[1,1] # + lhs*μ[i]
        mu  = ilhs*rhs + μ[i]
        μ[i,1] = mu + randn()*sqrt(ilhs)
    end
    # adjust for μ
    YCorr = YCorr .- μ;

    # sample locus effects
    for locus=1:p
        # unadjust for locus
        YCorr = YCorr .+ Alpha[:,locus]*Q[:,locus]'
        Rhs = iR*YCorr*Q[:,locus]
        Lhs = dot(Q[:,locus],Q[:,locus])*iR + iG 
        #println(YCorr[1,:])
        #println("Rhs",Rhs)
        #println("Lhs",Lhs)

        for i=1:nTr        
            lhs  = Lhs[i,i]
            ilhs = 1/lhs
            rhs  = (Rhs[i] - Lhs[i,:]*Alpha[:,locus])[1,1] # + lhs*μ[i]
            #println("lhs rhs",lhs," ",rhs)
            mu   = ilhs*rhs + Alpha[i,locus] 
            Alpha[i,locus] = mu + randn()*sqrt(ilhs)
        end
        # adjust for locus j
        YCorr = YCorr .- Alpha[:,locus]*Q[:,locus]'
        #println("effects", Alpha[:,locus])
    end
    SR = YCorr*YCorr'
    SG = Alpha*Alpha'

    R = rand(InverseWishart(νR + n, PR + SR))
    G = rand(InverseWishart(νG + p, PG + SG))
    RMean  += (R  - RMean)/iter  
    GMean  += (G  - GMean)/iter 
    if iter%100==0
        println("at sample: ",iter,"\n")
        println("GMMean",GMean)
    end
    #println(RMean)
end

In [ ]:
G0

In [ ]:
GMean